<a href="https://colab.research.google.com/github/kinjaljoshi/llm_finetune_peft/blob/main/llm_finetune_vs_Peft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Full fine tuning

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate -U

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

###Load Dataset

In [2]:
#load dataset
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Sample 1:
{'id': 'train_0', 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.", 'summary': "Mr

In [3]:

# Print a few samples
for i in range(5):  # Print 5 samples from the dataset
    print(f"Sample {i+1}:")
    print(dataset['train'][i])
    print("-" * 100)


Sample 1:
{'id': 'train_0', 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.", 'summary': "Mr

##Load LLM and tokenizer

In [22]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
model_name='google/flan-t5-base'
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

###Number of trainable parameters

In [7]:
def trainable_params(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in base_model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(trainable_params(base_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [14]:
#ideally for larger prompts we should use sliding window approach for tokenization so as to
#avoid lossing context context window for FLAN-T5 is 512
def tokenize_function(samples):
    start_prompt = 'Summarize the conversation.\n\n'
    end_prompt = '\n\nSummary:\n\n\n'
    prompt = [start_prompt + dialogue + end_prompt for dialogue in samples["dialogue"]]

    inputs = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(samples["summary"], padding="max_length", truncation=True, return_tensors="pt")


    # Move to CUDA if available
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        labels = {k: v.to("cuda") for k, v in labels.items()}

    samples['input_ids'] = inputs["input_ids"]
    samples['labels'] = labels["input_ids"]


    return samples

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

print(f"Training Dataset: {tokenized_datasets['train'].shape}")
print(f"Validation Dataset: {tokenized_datasets['validation'].shape}")
print(f"Testing Dataset: {tokenized_datasets['test'].shape}")

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Training Dataset: (12460, 2)
Validation Dataset: (500, 2)
Testing Dataset: (1500, 2)


In [23]:
from transformers import TrainingArguments, Trainer
output_dir = '/content/finetuned'
base_model.to('cuda')

training_args = TrainingArguments(
    output_dir=output_dir, #Specifies the directory where all training artifacts (checkpoints, logs, etc.) will be saved
    learning_rate=1e-3,#Defines the learning rate for the optimizer.Setting high learning rate
    num_train_epochs=3, #Defines how many epo chs (full passes through the training dataset) the model will train for.
    report_to='none', # Disables logging to external services like WandB, TensorBoard, or Comet.
    weight_decay=0.01, # Implements L2 regularization, preventing overfitting by penalizing large weight values
)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

trainer.train()

Step,Training Loss
500,0.473500
1000,0.102700
1500,0.105900


Step,Training Loss
500,0.473500
1000,0.102700
1500,0.105900
2000,0.087500
2500,0.084500
3000,0.087200
3500,0.078400
4000,0.075400
4500,0.075200


TrainOutput(global_step=4674, training_loss=0.12802929729287013, metrics={'train_runtime': 854.5116, 'train_samples_per_second': 43.744, 'train_steps_per_second': 5.47, 'total_flos': 2.559622983450624e+16, 'train_loss': 0.12802929729287013, 'epoch': 3.0})

##**Training a fully fine-tuned version of the model would take a few hours on a GPU.**

##Test after full finetune

In [24]:
from transformers import GenerationConfig
trained_model = base_model
model_name='google/flan-t5-base'
base_old_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

index = 20
dialogue = dataset['test'][index]['dialogue']
human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the conversation..

{dialogue}

Summary:


"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')
trained_model.to('cuda')
base_old_model.to('cuda')

# Generate Summary
base_output = base_old_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
base_decoded = tokenizer.decode(base_output[0], skip_special_tokens=True)

trained_output = trained_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
trained_decoded = tokenizer.decode(trained_output[0], skip_special_tokens=True)

print('-' * 100)
print(f'Label:\n{human_summary}')
print('-' * 100)
print(f'Base Model Output:\n{base_decoded}')
print('-' * 100)
print(f'Trained model Output:\n{trained_decoded}')

----------------------------------------------------------------------------------------------------
Label:
#Person1# thinks #Person2# has chicken pox and warns #Person2# about the possible hazards but #Person2# thinks it will be fine.
----------------------------------------------------------------------------------------------------
Base Model Output:
The person is scratching so much that he can't stand it anymore.
----------------------------------------------------------------------------------------------------
Trained model Output:
#Person2# scratched so much because #Person2# feels lightheaded and weak. #Person1# thinks #Person2# has chicken pox and asks #Person2# to get away from him. #Person2# thinks #Person2# has a biohazard and will take an oatmeal bath.


In [28]:
from peft import LoraConfig, get_peft_model

# Define LoRA Configuration
lora_config = LoraConfig(
    r=8,  # Low-rank matrix size
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    target_modules=["q", "v"]  # Target attention layers
)

# Apply LoRA to the model
peft_model = get_peft_model(base_old_model, lora_config)

# Print trainable parameters
print(trainable_params(peft_model))


trainable model parameters: 884736
all model parameters: 248462592
percentage of trainable model parameters: 0.36%


In [29]:
from transformers import TrainingArguments, Trainer
output_dir = '/content/peft_finetuned'
base_model.to('cuda')

training_args = TrainingArguments(
    output_dir=output_dir, #Specifies the directory where all training artifacts (checkpoints, logs, etc.) will be saved
    learning_rate=1e-3,#Defines the learning rate for the optimizer.Setting high learning rate
    num_train_epochs=3, #Defines how many epo chs (full passes through the training dataset) the model will train for.
    report_to='none', # Disables logging to external services like WandB, TensorBoard, or Comet.
    weight_decay=0.01, # Implements L2 regularization, preventing overfitting by penalizing large weight values decay*sqr(weight)
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

trainer.train()

Step,Training Loss
500,1.499200
1000,0.123200
1500,0.116000
2000,0.112300
2500,0.109900
3000,0.107200
3500,0.105800
4000,0.104000
4500,0.102900


TrainOutput(global_step=4674, training_loss=0.2585112833293218, metrics={'train_runtime': 877.3603, 'train_samples_per_second': 42.605, 'train_steps_per_second': 5.327, 'total_flos': 2.56978252726272e+16, 'train_loss': 0.2585112833293218, 'epoch': 3.0})

##Test After PEFT training

In [30]:

index = 20
dialogue = dataset['test'][index]['dialogue']
human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the conversation..

{dialogue}

Summary:


"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')
trained_model.to('cuda')
base_old_model.to('cuda')

# Generate Summary
base_output = base_old_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
base_decoded = tokenizer.decode(base_output[0], skip_special_tokens=True)

trained_output = trained_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
trained_decoded = tokenizer.decode(trained_output[0], skip_special_tokens=True)

peft_output = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_decoded = tokenizer.decode(peft_output[0], skip_special_tokens=True)


print('-' * 100)
print(f'Label:\n{human_summary}')
print('-' * 100)
print(f'Base Model Output:\n{base_decoded}')
print('-' * 100)
print(f'Trained model Output:\n{trained_decoded}')

print('-' * 100)
print(f'PEFT model Output:\n{peft_decoded}')

----------------------------------------------------------------------------------------------------
Label:
#Person1# thinks #Person2# has chicken pox and warns #Person2# about the possible hazards but #Person2# thinks it will be fine.
----------------------------------------------------------------------------------------------------
Base Model Output:
#Person2# feels itchy and feels weak. #Person1# thinks #Person2# has chicken pox and #Person2# thinks #Person2# is a biohazard. #Person2# thinks #Person2# is a biohazard. #Person2# thinks #Person1# is a biohazard.
----------------------------------------------------------------------------------------------------
Trained model Output:
#Person2# feels itchy and weak and feels itchy. #Person1# thinks #Person2# may be contagious and #Person2#'s a biohazard and #Person2# thinks #Person2# is a biohazard. #Person2# will go take an oatmeal bath.
---------------------------------------------------------------------------------------------------

Overall, LoRA provides the best trade-off between computational efficiency and fine-tuning effectiveness. It drastically reduces the cost of adapting large models while maintaining strong task performance. Compared to full fine-tuning, LoRA makes LLM fine-tuning more accessible, modular, and scalable, ensuring that organizations and researchers can adapt models to specific tasks without requiring massive computing resources.